In [23]:
!pip install ftfy
!pip install -q transformers
!pip install sentencepiece
!pip install torch_optimizer
!pip install tensorflow-addons
!pip install torch torchvision

In [24]:
#pip install -q transformers
#pip install ftfy
#pip install tensorflow-addons
#pip install torch-optimizer

In [25]:
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.utils import pad_sequences
from torch.nn import BCEWithLogitsLoss, BCELoss
from transformers import *
import os, re, string
import numpy as np
import pickle, statistics
from tensorflow.keras.layers import *
from ftfy import fix_text
from matplotlib import pyplot
from sklearn.metrics import *
from tqdm import tqdm, trange
import warnings
import pickle
import tensorflow as tf
from sklearn.utils import shuffle
import tensorflow_addons as tfa
import torch_optimizer as optim
from sklearn.metrics import *
warnings.filterwarnings("ignore")
from transformers import logging
logging.set_verbosity_error()

In [32]:
# from google.colab import drive
# drive.mount('/content/gdrive')

path_train_en ='./Dataset/En_Train.txt'
path_test_en ='./Dataset/En_Test.txt'

In [33]:
class Review():
    def __init__(self,stt,text,label):
        self.stt = stt
        self.labels = label
        self.orginalText = text

def read_files(filename):
    documents = list()
    count = 0
    with open(filename,'r',encoding='utf8') as file:
        text1 = file.read()
        for item in text1.split('\n'):
            if item != '':
                item = item.strip()
                if count == 0:
                    stt = fix_text(item)
                    count += 1
                elif count == 1:
                    text = fix_text(item)
                    count += 1
                elif count == 2:
                    labels = fix_text(item).replace("STYLE&OPTIONS", "STYLE_OPTIONS")
                    count = 0
                    reviewtemp = Review(stt, text, labels)
                    documents.append(reviewtemp)
    return documents

In [34]:
import string,re
def clean_doc(text):
  text = text.lower()
  text = text.translate(text.maketrans('', '', string.punctuation))
  text = re.sub(r"[0-9]+", " num ", text)
  text = re.sub('\\s+',' ',text)
  text = text.strip()
  return text

In [35]:
# Function convert label to vector to fit model. 
# Consider each aspect as one vector. Example vector for an aspect not being mentioned is: [1,0,0,0]
# Vector for mentioned positive aspect: [0,1,0,0]. Vector for mentioned. negative aspect : [0,0,0,1]
def to_category_vector_aspect_polarity(label):
    vector = np.zeros(4).astype(np.float64)
    if label.strip() != '':
        if 'positive' in label:
            vector[1] = 1.0
        elif 'neutral' in label:
            vector[2] = 1.0
        elif 'negative' in label:
            vector[3] = 1.0
        else:
            vector[0] = 1.0
    else:
        vector[0] = 1.0
    return vector

import re
def create_output_aspect_polarity(labels,categories,list_output):
    for i,category in enumerate(categories):
        if category in labels:
            try:
                output = re.findall('('+category+',\s(positive|negative|neutral))',labels)[0][0]
            except:
                output = ''
            list_output[i].append(to_category_vector_aspect_polarity(output))
        else:
            list_output[i].append(to_category_vector_aspect_polarity(''))
    return list_output

In [36]:
def read_data_train(languages):
    data  = []
    if "en" in languages:
      # English
      for review in read_files(path_train_en):
          data.append(review)

    print(len(data))
    return data

def read_data_test(target_language):
    data = []
    if "en" in target_language:
      for review in read_files(path_test_en):
          data.append(review)

    print(len(data))    
    return data

In [37]:
listLabel = "RESTAURANT#GENERAL,SERVICE#GENERAL,FOOD#QUALITY,FOOD#STYLE_OPTIONS,DRINKS#STYLE_OPTIONS,DRINKS#PRICES,RESTAURANT#PRICES,RESTAURANT#MISCELLANEOUS,AMBIENCE#GENERAL,FOOD#PRICES,LOCATION#GENERAL,DRINKS#QUALITY"
categories = listLabel.split(',')

x_train = []
y_train = []

output_train1 = []
output_train2 = []
output_train3 = []
output_train4 = []
output_train5 = []
output_train6 = []
output_train7 = []
output_train8 = []
output_train9 = []
output_train10 = []
output_train11 = []
output_train12 = []

list_output_train = list()
list_output_train.append(output_train1)
list_output_train.append(output_train2)
list_output_train.append(output_train3)
list_output_train.append(output_train4)
list_output_train.append(output_train5)
list_output_train.append(output_train6)
list_output_train.append(output_train7)
list_output_train.append(output_train8)
list_output_train.append(output_train9)
list_output_train.append(output_train10)
list_output_train.append(output_train11)
list_output_train.append(output_train12)

y_train_category = []
for review in read_data_train("en"):
    x_train.append(clean_doc(review.orginalText))
    y_train = create_output_aspect_polarity(review.labels, categories,list_output_train)

x_test = []
y_test_category = []
for review in read_data_test("en"):
    x_test.append(clean_doc(review.orginalText))

print(len(x_train), len(x_test))

1708
587
1708 587


In [38]:
# RobERTA MODEL
# Reference: https://arxiv.org/abs/1907.11692
language = "en"
print("Load English model")
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name, do_lower_case=False)
config = RobertaConfig.from_pretrained(model_name, output_hidden_states=True)
transformer_model = TFRobertaModel.from_pretrained(model_name, config = config,from_pt=True)


Load English model


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

2022-09-24 16:37:11.765082: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [39]:
train_encodings = tokenizer(x_train, truncation=True, padding=True)
print(len(train_encodings["input_ids"]))
print(len(train_encodings["attention_mask"]))
max_length = len(train_encodings["input_ids"][0])
print(max_length)

1708
1708
74


In [42]:
train_ids = np.array(train_encodings["input_ids"])
train_masks = np.array(train_encodings["attention_mask"])

In [43]:
test_encodings = tokenizer(x_test, max_length=max_length, truncation=True, padding="max_length")
print(max_length)
test_ids = np.array(test_encodings["input_ids"])
test_masks = np.array(test_encodings["attention_mask"])

74


In [48]:
import pickle
import tensorflow as tf
import tensorflow_addons as tfa

def build_model(max_length=max_length):
      input_ids_in = tf.keras.layers.Input(shape=(max_length,), name='input_ids', dtype='int32')
      input_masks_in = tf.keras.layers.Input(shape=(max_length,), name='input_mask', dtype='int32') 

      outputs = transformer_model(input_ids_in,attention_mask = input_masks_in)[0]

      cls_token = outputs[:,0,:]

      output1 = tf.keras.layers.Dense(4,name="output1", activation='softmax')(cls_token)
      output2 = tf.keras.layers.Dense(4,name="output2", activation='softmax')(cls_token)
      output3 = tf.keras.layers.Dense(4,name="output3", activation='softmax')(cls_token)
      output4 = tf.keras.layers.Dense(4,name="output4", activation='softmax')(cls_token)
      output5 = tf.keras.layers.Dense(4,name="output5", activation='softmax')(cls_token)
      output6 = tf.keras.layers.Dense(4,name="output6", activation='softmax')(cls_token)
      output7 = tf.keras.layers.Dense(4,name="output7", activation='softmax')(cls_token)
      output8 = tf.keras.layers.Dense(4,name="output8", activation='softmax')(cls_token)
      output9 = tf.keras.layers.Dense(4,name="output9", activation='softmax')(cls_token)
      output10 = tf.keras.layers.Dense(4,name="output10", activation='softmax')(cls_token)
      output11 = tf.keras.layers.Dense(4,name="output11", activation='softmax')(cls_token)
      output12 = tf.keras.layers.Dense(4,name="output12", activation='softmax')(cls_token)

      model = tf.keras.Model(inputs=[input_ids_in,input_masks_in], outputs=[output1,output2,output3,output4,output5,output6,output7,output8,output9,output10,output11,output12])

      opt = tf.keras.optimizers.Adam(learning_rate=5e-5,amsgrad=True)
      checkpoint_filepath = './checkpoint'
      model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
          filepath=checkpoint_filepath,
          save_weights_only=True,
          monitor='loss',
          mode='min',
          save_best_only=True)
      #opt = tfa.optimizers.AdamW(weight_decay = 0.01, learning_rate = 5e-5)
      loss = tf.keras.losses.CategoricalCrossentropy()
      metric = tf.keras.metrics.CategoricalAccuracy('accuracy')
      model.compile(loss={'output1':loss,'output2':loss,'output3':loss,'output4':loss,'output5':loss,'output6':loss,'output7':loss,'output8':loss,'output9':loss,'output10':loss,'output11':loss,'output12':loss},optimizer=opt, metrics = [metric])
      print(model.summary())
        
      list_total_Y_train  = [np.array(list_output_train[0]),np.array(list_output_train[1]),np.array(list_output_train[2]),np.array(list_output_train[3]),np.array(list_output_train[4]),np.array(list_output_train[5]),np.array(list_output_train[6]),np.array(list_output_train[7]),np.array(list_output_train[8]),np.array(list_output_train[9]),np.array(list_output_train[10]),np.array(list_output_train[11])]
              
      history = model.fit([train_ids,np.array(train_masks)], list_total_Y_train,batch_size=32, epochs=50,callbacks=[model_checkpoint_callback])
      model.load_weights(checkpoint_filepath)
      return model

In [ ]:
# Save result in folder Result
import operator, random
path_save = './Result/'

seed = 42
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
random.seed(seed)
np.random.seed(seed)
model = build_model()
count = 0
textPrint = ''
textREsult = []
for index,item in enumerate(test_ids):
    predicted = model.predict([np.expand_dims(item, axis=0),np.expand_dims(np.array(test_masks[index]), axis=0)])
    s = ''
    for i, predict in enumerate(predicted):
        index2, value = max(enumerate(predict[0]), key=operator.itemgetter(1))
        if index2 == 1:
            s+= '{' + str(categories[i]) + ', positive}, '
        elif index2 == 2:
            s+= '{' + str(categories[i]) + ', neutral}, '
        elif index2 == 3:
            s+= '{' + str(categories[i]) + ', negative}, '
    if s.strip() == "":
      s = "{RESTAURANT#GENERAL, neutral}, "
    textPrint += '#' + str(count +1)+'\n'
    textPrint += x_test[index] + '\n'
    textPrint += s[:len(s)-2] +'\n\n'
    count +=1
with open(path_save + 'MonoRoBERTa.txt','w',encoding = 'utf8') as file:
    file.write(textPrint)
with open('MonoRoBERTa.txt','w',encoding = 'utf8') as file:
    file.write(textPrint)
print("Done")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 74)]         0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 74)]         0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 el)                            thPoolingAndCrossAt               'input_mask[0][0]']             
                                tentions(last_hidde                                               
                                n_state=(None, 74,                                            

In [ ]:
# Evaluation model
!cp "EvaluationSystemByFile.py" "EvaluationSystemByFile.py"
!cp 'Dataset/En_Test.txt' "En_Test.txt"

print("Result of model on Micro F1-score")
!python EvaluationSystemByFile.py "En_Test.txt" "MonoRoBERTa.txt"

In [ ]:
#Save model to path
path_save_model = './Model/'

model.save(path_save_model + "RoBertTa_1.h5")

In [ ]:
# Reference
# 1. https://aclanthology.org/D18-1139.pdf
# 2. https://www.researchgate.net/publication/335872705_Joint_Learning_for_Aspect_Category_Detection_and_Sentiment_Analysis_in_Chinese_Reviews
# 3. 